# Wildfire Smoke Controls on Gross Primary Production in Central Canada

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.stats import pearsonr

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# get Sam's stepwise selection function
import statsmodels.api as sm
%run ../lab02/Tutorial_2_2021_functions2.ipynb

# show full dataframes
pd.set_option('max_columns', None)

In [2]:
# get data
data_in = pd.read_csv("drf_timeseries.csv", parse_dates=True, index_col=0)

In [ ]:
data_in["datetime"] = pd.to_datetime(data_in["datetime"])

## Select only the relevant months

This study is really only concerned with what goes on between May and September, as the vast majority of primary production (trees photosynthesizing/consuming CO2) and wildfire activity happen in the summer

In [ ]:
# select just the growing/wildfire season (may-sept, inclusive)
growing_season = pd.DataFrame()
#for month in [5,6,7,8,9]:
for month in [1,2,3,4,5,6,7,8,9,10,11,12]:
    growing_season = growing_season.append(data_in[pd.to_datetime(data_in['datetime']).dt.month == month])
growing_season = growing_season.sort_values(by='datetime')
#growing_season["year"] = pd.to_datetime(data_in['datetime']).dt.year

## Find which measurements have the most complete data

In [ ]:
# check data for completeness
def check_complete(data_in):
    """
    Prints out the percentage of non-NaN values in a dataset
    """
    data_len = np.shape(data_in)[0]
    for key in data_in.keys():
        if key != "datetime":
            not_nans = np.shape((data_in[np.isnan(data_in[key]) == False]))[0]
            completeness = not_nans / data_len * 100
            print(f"{key}:  {round(completeness,2)} %")
    return None
 

def keep_complete(data_in, thres):
    """
    returns a dataframe that contains a percentage of non-NaNs above
    a specified threshhold
    """
    data_out = pd.DataFrame()
    data_out["datetime"] = data_in["datetime"]
    data_len = np.shape(data_in)[0]
    for key in data_in.keys():
        if key != "datetime":
            not_nans = np.shape((data_in[np.isnan(data_in[key]) == False]))[0]
            completeness = not_nans / data_len * 100
            if completeness >= thres:
                data_out[key] = data_in[key]
    return data_out

In [ ]:
check_complete(growing_season)

Pretty dismal. We need the AOD data, so set a cutoff at 5%. We can hopefully interpolate the rest

## Interpolation

Now, go through each individual measurement and hand-tune the pandas interpolation scheme to come up with something realistic (If this can't be achieved, toss the whole column). There is a balance here, we are trying to maximize coverage where all columns are finite (not NaNs, so we can perform PCA on as big of dataset as possible), without dangerously extending data beyond what is physical.

### AOD Data

In [ ]:
data = keep_complete(growing_season, 4.9)
check_complete(data) # see if that worked

In [ ]:
# get all the AOD columns that survived the purge
aod_list = []
for key in data.keys():
    if "AOD" in key:
        aod_list.append(key)
        print(key)

In [ ]:
data = keep_complete(growing_season, 4.9) # reset it
# peek at the raw AOD data. How big of gaps am I willing to try to fill?
fig, ax = plt.subplots(figsize=(15,4))
data[aod_list][24000:30000].plot(alpha=0.5, ax=ax);

In [ ]:
data[aod_list] = data[aod_list].interpolate(method='time', limit=10, limit_direction='both')

fig, ax = plt.subplots(figsize=(15,4))
data[aod_list][24000:30000].plot(alpha=0.5, ax=ax);

In [ ]:
# interpolation method I settled on (subject to change). Try messing around with this, 
# lots of methods available, most arent appropriate
data["AOD_500nm"].interpolate(method='slinear', limit=50, limit_direction='both').plot();

In [ ]:
# now apply the interpolation scheme to all AOD sets
for key in aod_list:
    data[key] = data[key].interpolate(method='slinear', limit=500, limit_direction='both')
data.plot('datetime', aod_list)

In [ ]:
# Blanket interpolate all data using the same scheme (this is a bad idea)
data_interp = data.interpolate(method='linear')#.dropna()
data_interp.plot()

In [ ]:
# after we finish messing about with interpolations, drop all rows that still have missing data
data_complete = data_interp.dropna()
data_complete

In [ ]:
###################################################
# choose which version of the dataset to use here #
###################################################

#data = data_complete.drop("datetime", axis=1)
data = data_interp.drop("datetime", axis=1).dropna()

## Part 2: Try PCA 

In [ ]:
#data = data_complete.drop("datetime", axis=1)
data = data_interp.drop("datetime", axis=1).dropna()

n_modes = np.min(np.shape(data))
pca = PCA(n_components = n_modes)
PCs = pca.fit_transform(data)
eigvecs = pca.components_
fracVar = pca.explained_variance_ratio_

In [ ]:
#plot fraction of variance explained by each mode
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.scatter(range(len(fracVar)),fracVar)
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by All Modes')

plt.subplot(1,2,2)
n_modes_show = 10
plt.scatter(range(n_modes_show),fracVar[:n_modes_show])
plt.xlabel('Mode Number')
plt.ylabel('Fraction Variance Explained')
plt.title('Variance Explained by First ' + str(n_modes_show) + ' Modes')

plt.tight_layout()

plt.show()

In [ ]:
plt.plot(PCs[...,:4]);

PCA works properly. What are the biggest contributors to the leading modes?

## Multiple Linear Regression

Perform MLR on the PCs to create a predictive model with inputs

**PCs $\rightarrow$ CO2 Fluxes**

or, if we do rotated PCA:

**smoke, PCs $\rightarrow$ CO2 Fluxes**

## 1) MLR with the PCs

In [ ]:
# perform MLR with the columns of the PCs
### normalize the PCs first ###
### only use a few PCs
### separate carbon flux outputs ###


# assign predictors and predictands
X = pd.DataFrame(PCs)
Y = data["FC"]

# Do MLR
model = LinearRegression().fit(X, Y)
ypred_MLR = model.predict(X)  # y predicted by MLR
#intercept_MLR = model.intercept_[0]  # intercept predicted by MLR
coef_MLR = model.coef_.flatten()  # regression coefficients in MLR model
R2_MLR = model.score(X, Y)  # R-squared value from MLR model

# Display the results
for i, coef in enumerate(coef_MLR):
    print(f"PC{i}: {coef}")

## 2) MLR With the Regular Dataframe

In [ ]:
# assign predictors and predictands
X = data.drop("FC", axis=1)
Y = data["FC"]

# Do MLR
model = LinearRegression().fit(X, Y)
ypred_MLR = model.predict(X)  # y predicted by MLR
#intercept_MLR = model.intercept_[0]  # intercept predicted by MLR
coef_MLR = model.coef_.flatten()  # regression coefficients in MLR model
R2_MLR = model.score(X, Y)  # R-squared value from MLR model

# Display the results
for col, coef in zip(data.keys(), coef_MLR):
    print(f"{col}: {coef}")

## To Do

- Interpolate all columns, eyeball-optimizing between coverage and faithful interpolated values
- chop out dates and outputs, perform PCA on everything else
- try step 2 again, this time grouping on smoke level. Are the leading modes different?


- Try it again with rotated PCA
- Write up the presentation
